# Convolution Network

In [4]:
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle

## Helper functions

In [5]:
def unpickle(file):
    fo = open(file, 'rb')
    data = pickle.load(fo, encoding='latin1')
    fo.close
    return data


def clean(data):
    # We need to fix shape
    imgs = data.reshape(data.shape[0], 3, 32, 32).astype(np.float32)
    # Naive grayscala conversion
    grayscala_imgs = imgs.mean(1)
    # For faster processing we only need 24x24 images
    cropped_imgs = grayscala_imgs[:, 4:28, 4:28]
    img_data = cropped_imgs.reshape(data.shape[0], -1)
    img_size = np.shape(img_data)[1]
    means = np.mean(img_data, axis=1)
    meansT = means.reshape(len(means), 1)
    stds = np.std(img_data, axis=1)
    stdsT = stds.reshape(len(stds), 1)
    adj_stds = np.maximum(stdsT, 1.0 / np.sqrt(img_size))
    normalized = (img_data - meansT) / adj_stds
    return normalized
    
    
def load_images(path):
    names = unpickle(path + 'batches.meta')['label_names']
    data, labels = [], []
    for i in range(1, 6):
        filename = path + 'data_batch_' + str(i)
        batch_data = unpickle(filename)
        if len(data) > 0:
            data = np.vstack([data, batch_data['data']])
            labels = np.hstack([labels, batch_data['labels']])
        else:
            data = batch_data['data']
            labels = batch_data['labels']
    data = clean(data)
    return names, data, labels

## Load data

In [6]:
names, data, labels = load_images('../datasets/cifar-10-batches-py/')

## Prepare weights

In [8]:
x = tf.placeholder(tf.float32, [None, 24*24])
y = tf.placeholder(tf.float32, [None, len(names)])

W1 = tf.Variable(tf.random_normal([5, 5, 1, 64]))
b1 = tf.Variable(tf.random_normal([64]))

W2 = tf.Variable(tf.random_normal([5, 5, 64, 64]))
b2 = tf.Variable(tf.random_normal([64]))

# Fuly connected layer
W3 = tf.Variable(tf.random_normal([6*6*64, 1024]))
b3 = tf.Variable(tf.random_normal([1024]))

W_out = tf.Variable(tf.random_normal([1024, len(names)]))
b_out = tf.Variable(tf.random_normal([len(names)]))

## Some helper functions for creating ANN

In [12]:
def conv_layer(x, W, b):
    conv = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
    conv_with_b = tf.nn.bias_add(conv, b)
    return tf.nn.relu(conv_with_b)


def maxpool_layer(conv, k=2):
    return tf.nn.max_pool(conv, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')


def model():
    x_reshaped = tf.reshape(x, shape=[-1, 24, 24, 1])

    conv_out1 = conv_layer(x_reshaped, W1, b1)
    maxpool_out1 = maxpool_layer(conv_out1)
    norm1 = tf.nn.lrn(maxpool_out1, 4, bias=1.0, alpha=0.001/9.0, beta=0.75)

    conv_out2 = conv_layer(norm1, W2, b2)
    maxpool_out2 = maxpool_layer(conv_out2)
    norm2 = tf.nn.lrn(maxpool_out2, 4, bias=1.0, alpha=0.001/9.0, beta=0.75)    
    
    maxpool_reshaped = tf.reshape(norm2, [-1, W3.get_shape().as_list()[0]])
    local = tf.add(tf.matmul(maxpool_reshaped, W3), b3)
    local_out = tf.nn.relu(local)
    
    return tf.add(tf.matmul(local_out, W_out), b_out)

## Build network model

In [14]:
model_op = model()

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model_op, labels=y))
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

correct_pred = tf.equal(tf.argmax(model_op, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

## Train the classifier

In [18]:
with tf.Session() as sess:
    print('Dataset size: {}'.format(np.shape(data)[0]))
    sess.run(tf.global_variables_initializer())
    onehot_labels = tf.one_hot(labels, len(names), on_value=1.0, off_value=0.0, axis=-1)
    onehot_vals = sess.run(onehot_labels)
    batch_size = len(data) // 200
    print('batch_size', batch_size)
    for j in range(10):
        print('Epoch', j)
        for i in range(0, len(data), batch_size):
            batch_data = data[i:i+batch_size, :]
            batch_onehot_vals = onehot_vals[i:i+batch_size, :]
            _, accuracy_val = sess.run([train_op, accuracy], feed_dict={x:batch_data, y: batch_onehot_vals})
            if i % 10000 == 0:
                print(i, accuracy_val)
    print('Done. Accuracy: {}'.format(accuracy_val))

Dataset size: 50000
batch_size 250
Epoch 0
0 0.088
10000 0.228
20000 0.216
30000 0.228
40000 0.244
Epoch 1
0 0.256
10000 0.32
20000 0.296
30000 0.268
40000 0.244
Epoch 2
0 0.24
10000 0.308
20000 0.32
30000 0.276
40000 0.284
Epoch 3
0 0.272
10000 0.304
20000 0.324
30000 0.308
40000 0.304
Epoch 4
0 0.304
10000 0.3
20000 0.308
30000 0.312
40000 0.32
Epoch 5
0 0.348
10000 0.3
20000 0.312
30000 0.308
40000 0.328
Epoch 6
0 0.32
10000 0.288
20000 0.336
30000 0.332
40000 0.34
Epoch 7
0 0.352
10000 0.284
20000 0.328
30000 0.324
40000 0.328
Epoch 8
0 0.348
10000 0.296
20000 0.348
30000 0.368
40000 0.332
Epoch 9
0 0.356
10000 0.328
20000 0.352
30000 0.376
40000 0.34
Done. Accuracy: 0.3720000088214874
